- 2306xx RosettaFold in Jupyter!
- 230628-29 added a way to get from Rfs into XYZ, verified, all rmsd work!

In [1]:
angle = 90

import torch
from numpy import *
import numpy as np
from icecream import ic
import sys
ic.configureOutput(outputFunction=lambda *a: print(*a, file=sys.stderr))
ic.configureOutput(prefix='> ')
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

import torch
from torch import tensor
import numpy as np


from Bio import PDB
from Bio.PDB import PDBIO
import warnings
warnings.filterwarnings('ignore', '.*Invalid or missing.*',)
warnings.filterwarnings('ignore', '.*with given element *',)
warnings.filterwarnings('ignore', '.*END*',)
from icecream import ic
ic.configureOutput(prefix='')
import sys
ic.configureOutput(outputFunction=lambda *a: print(*a, file=sys.stdout))
from Bio.PDB.vectors import Vector, rotmat
import torch

def rigidFrom3PointsTT(x1, x2, x3, mode = 0):
    """
    with no normalization
    """
    v1 = x3 - x2 # 0
    if mode: # 1
        v1 = x2 - x3
    v2 = x1 - x2

    e1 = v1 / torch.linalg.norm(v1)
    u2 = e1 * torch.dot(v2, e1)
    e2 = v2 - u2
    e2 = e2 / np.linalg.norm(e2)
    e3 = torch.cross(e1, e2) #.reshape((3,)), e2.reshape((3,))).reshape((3,1))
    R = torch.stack((e1,e2,e3), dim=-1)
    t = x2
    return R, t, e1, e2, e3

def rnd(x):
    return np.round(x, decimals=3)




![](./a24.jpg)

# Global frame

In [148]:
parser = PDB.PDBParser()
fn = "/Users/magnus/work/src/rf-structure-module/examples/4GXY_min_at0.pdb"
struct = parser.get_structure('',fn)
chain = struct[0]['A'] ## to fix

x1 = tensor(struct[0]['A'][2]['OP1'].get_coord())
x2 = tensor(struct[0]['A'][2]['P'].get_coord())
x3 = tensor(struct[0]['A'][2]['OP2'].get_coord())

ic(x1, x2, x3)
mode = 0
R, t, e1, e2, e3 = rigidFrom3PointsTT(x1, x2, x3, mode) 
R, t

x1: tensor([ 0.1490, -1.1410, -0.9390])
x2: tensor([0., 0., 0.])
x3: tensor([-1.1230,  0.0040,  0.9760])


(tensor([[-0.7548, -0.3119,  0.5771],
         [ 0.0027, -0.8812, -0.4727],
         [ 0.6560, -0.3552,  0.6660]]),
 tensor([0., 0., 0.]))

In [149]:
fglobal = torch.eye(4)
fglobal[:3,:3] = R
fglobal[:3,3] = t
fglobal

tensor([[-0.7548, -0.3119,  0.5771,  0.0000],
        [ 0.0027, -0.8812, -0.4727,  0.0000],
        [ 0.6560, -0.3552,  0.6660,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

```python
# RosettaFoldNA
RTF0[seq_i,:,:]: tensor([[-0.7548, -0.3119,  0.5771,  0.0000],
                         [ 0.0027, -0.8812, -0.4727,  0.0000],
                         [ 0.6560, -0.3552,  0.6660,  0.0000],
                         [ 0.0000,  0.0000,  0.0000,  1.0000]])

 ```

In [150]:
fglobal @ tensor([-0.7319, 1.2920, 0.000, 1]) # OP1
#ATOM     22  OP1   G A   2       0.149  -1.141  -0.939  0.00  0.00           O  

tensor([ 0.1495, -1.1405, -0.9391,  1.0000])

In [151]:
fglobal @ tensor([-0.4948, -0.8559,  1.2489, 1]) #  O5' # wrong one !!!!!!!!!
#  51.384  77.570  40.867  0.00  0.00           O  
RMSD([51.3621, 77.4165, 40.9482], [51.384, 77.570, 40.867])

0.17502942609743846

In [152]:
def RMSD(X,Y):
    rmsd = 0
    rmsd = math.sqrt((X[0]-Y[0])*(X[0]-Y[0]) + (X[1]-Y[1])*(X[1]-Y[1]) + (X[2]-Y[2])*(X[2]-Y[2]))
    return rmsd;

In [153]:
#fglobal @ tensor([-0.56, -0.9690,  1.1350, 1]) #  O5' # wrong one !!!!!!!!!
# ATOM      3  O5'   G A   2       1.383   0.316   0.730  0.00  0.00           O  

In [154]:
fglobal @ tensor([1.4855, 0.000, 0.000, 1]) #OP2
#ATOM     23  OP2   G A   2      -1.123   0.004   0.976  0.00  0.00           O1-
#     48.878  77.258  41.113  0.00  0.00           O1-

tensor([-1.1212,  0.0040,  0.9745,  1.0000])

# Rotation frame

In [155]:
# calc torsion angle # rosettafoldrna
def th_dih_v(ab,bc,cd):
        def th_cross(a,b):
            a,b = torch.broadcast_tensors(a,b)
            return torch.cross(a,b, dim=-1)
        def th_norm(x,eps:float=1e-8):
            return x.square().sum(-1,keepdim=True).add(eps).sqrt()
        def th_N(x,alpha:float=0):
            return x/th_norm(x).add(alpha)

        ab, bc, cd = th_N(ab),th_N(bc),th_N(cd)
        n1 = th_N( th_cross(ab,bc) )
        n2 = th_N( th_cross(bc,cd) )
        sin_angle = (th_cross(n1,bc)*n2).sum(-1)
        cos_angle = (n1*n2).sum(-1)
        dih = torch.stack((cos_angle,sin_angle),-1)
        return dih

def th_dih(a,b,c,d):
        return th_dih_v(a-b,b-c,c-d)

x1 = tensor(struct[0]['A'][2]['OP1'].get_coord())
x2 = tensor(struct[0]['A'][2]['P'].get_coord())
x3 = tensor(struct[0]['A'][2]["O5'"].get_coord())
x4 = tensor(struct[0]['A'][2]["C5'"].get_coord())
th_dih(x1, x2, x3, x4)

tensor([ 0.4351, -0.9004])

In [156]:
def make_rotX_chi(angs, i, eps=1e-6):
    """
    Rotate about the x axis, for simplified version for single RNA looping over chain length.

    args:

       rf.py:414 in make_rotX_chi()
       angs: tensor([[[ 0.4348, -0.9005],
                   [ 0.9997,  0.0261]]])
                       cos     sin
       i: index

    rf.py:424 in make_rotX_chi()
    RTs: tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
                 [ 0.0000,  0.4348,  0.9005,  0.0000],
                 [ 0.0000, -0.9005,  0.4348,  0.0000],
                 [ 0.0000,  0.0000,  0.0000,  1.0000]])
    """
    angs2 = torch.squeeze(angs[:,i,:])
    ic(angs2)
    NORM = torch.linalg.norm(angs2, dim=-1) + eps
    RTs = torch.eye(4)

    RTs[1,1] = angs2[0]/NORM
    RTs[1,2] = -angs2[1]/NORM
    RTs[2,1] = angs2[1]/NORM
    RTs[2,2] = angs2[0]/NORM

    return RTs

angs = torch.tensor([[[ 0.4348, -0.9005],
                   [ 0.9997,  0.0261]]])
frot = make_rotX_chi(angs, 0)

angs2: tensor([ 0.4348, -0.9005])


tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.4348,  0.9005,  0.0000],
        [ 0.0000, -0.9005,  0.4348,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

```python
# RosettaFoldNA
make_rotX_chi(alphas[:,:,2,:], seq_i): tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
                                               [ 0.0000,  0.4348,  0.9005,  0.0000],
                                               [ 0.0000, -0.9005,  0.4348,  0.0000],
                                               [ 0.0000,  0.0000,  0.0000,  1.0000]])
```

![](./rotX.jpg)

# Local frame

In [157]:
op1 = tensor([-0.7319, 1.2920, 0.000]) # OP1 ideal
p = tensor([0.0, 0.0, 0.0])
o5p = tensor([-0.4948, -0.8559,  1.2489]) #O5' ideal 
mode = 0
Rl, tl, e1, e2, e3 = rigidFrom3PointsTT(op1, p, o5p, mode) 
Rl, tl

(tensor([[-0.3106, -0.6221, -0.7187],
         [-0.5373,  0.7386, -0.4071],
         [ 0.7841,  0.2597, -0.5637]]),
 tensor([0., 0., 0.]))

```
RTs_by_torsion[i,2,:3,:3] = make_frame(
            xyzs_in_base_frame[i,3,:3] - xyzs_in_base_frame[i,1,:3], # P->O5'
            xyzs_in_base_frame[i,0,:3] - xyzs_in_base_frame[i,1,:3]  # P<-OP1
        )
        
RTs_by_torsion[i,2,:3,:3]: tensor([[-0.3106, -0.6221, -0.7187],
                                   [-0.5373,  0.7386, -0.4071],
                                   [ 0.7841,  0.2597, -0.5637]])

```

In [158]:
def make_frame(X, Y):  # from RF
        """Process ideal frames
        Args:
            vectors X Y, e.g.::

                    tensor([-0.4948, -0.8559,  1.2489]), tensor([-0.7319,  1.2920,  0.0000])

          tensor, e.g.::

           tensor([[-0.3106, -0.6221, -0.7187],
                   [-0.5373,  0.7386, -0.4071],
                   [ 0.7841,  0.2597, -0.5637]])"""

        Xn = X / torch.linalg.norm(X)
        Y = Y - torch.dot(Y, Xn) * Xn
        Yn = Y / torch.linalg.norm(Y)
        Z = torch.cross(Xn,Yn)
        Zn =  Z / torch.linalg.norm(Z)
        return torch.stack((Xn,Yn,Zn), dim=-1)
    
R1 = make_frame(o5p - p, op1 - p)

tensor([[-0.3106, -0.6221, -0.7187],
        [-0.5373,  0.7386, -0.4071],
        [ 0.7841,  0.2597, -0.5637]])

In [159]:
flocal = torch.eye(4)
flocal[:3,:3] = Rl
flocal[:3,3] = o5p
flocal

tensor([[-0.3106, -0.6221, -0.7187, -0.4948],
        [-0.5373,  0.7386, -0.4071, -0.8559],
        [ 0.7841,  0.2597, -0.5637,  1.2489],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

```python
self.RTs_in_base_frame[s_i,2,:]: tensor([[-0.3106, -0.6221, -0.7187, -0.4948],
                                         [-0.5373,  0.7386, -0.4071, -0.8559],
                                         [ 0.7841,  0.2597, -0.5637,  1.2489],
                                         [ 0.0000,  0.0000,  0.0000,  1.0000]])

```

```python
RTF1[seq_i,:,:]: tensor([[ 0.8545, -0.1407,  0.5000,  1.3611],
                         [ 0.1021, -0.8984, -0.4272,  0.1626],
                         [ 0.5093,  0.4161, -0.7533,  0.8112],
                         [ 0.0000,  0.0000,  0.0000,  1.0000]])

```

In [160]:
f1 = torch.einsum(
            'ij,jk,kl->il',
            fglobal, flocal, frot)
f1

tensor([[ 0.8545, -0.1407,  0.5000,  1.3611],
        [ 0.1021, -0.8984, -0.4272,  0.1626],
        [ 0.5093,  0.4160, -0.7534,  0.8112],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

In [208]:
f1 = fglobal @ flocal @ frot # alphaframe

tensor([[ 0.8545, -0.1407,  0.5000,  1.3611],
        [ 0.1021, -0.8984, -0.4272,  0.1626],
        [ 0.5093,  0.4160, -0.7534,  0.8112],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

In [175]:
c5pic = tensor((0.7289, 1.2185, 0.000, 1)) #[-7.3190e-01,  1.2920e+00,  0.0000e+00,  1.0000e+00])

tensor([0.7289, 1.2185, 0.0000, 1.0000])

In [176]:
# x1: tensor([ 0.1490, -1.1410, -0.9390]) ATOM     22  OP1   G A   2       0.149  -1.141  -0.939  0.00  0.00           O  

In [177]:
f1 @ c5pic # ATOM      2  C5'   G A   2       1.978  -0.634   1.601  0.00  0.00           C  

tensor([ 1.8126, -0.8578,  1.6893,  1.0000])

In [209]:
RMSD(f1 @ c5pic, [1.978,  -0.634,   1.601]) # 1.978  -0.634   1.601 pdb

0.29193290695565965

In [210]:
# X

In [211]:
c5pI = tensor([0.7289, 1.2185, 0.000]) # 
R2 = make_frame(
    c5pI, 
                tensor([-1.0, 0.0, 0.0]))
flocal2 = torch.eye(4)
flocal2[:3,:3] = R2
flocal2[:3,3] = c5pI
flocal2

tensor([[ 0.5134, -0.8582,  0.0000,  0.7289],
        [ 0.8582,  0.5134, -0.0000,  1.2185],
        [ 0.0000,  0.0000,  1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

```python
self.RTs_in_base_frame[s_i,3,:]: tensor([[ 0.5134, -0.8582,  0.0000,  0.7289],
                                         [ 0.8582,  0.5134, -0.0000,  1.2185],
                                         [ 0.0000,  0.0000,  1.0000,  0.0000],
                                         [ 0.0000,  0.0000,  0.0000,  1.0000]])

```    

In [219]:
angs = torch.tensor([[[ 0.4348, -0.9005],
                   [-0.8006, -0.5992]]])
frot2 = make_rotX_chi(angs, 1)

angs2: tensor([-0.8006, -0.5992])


tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.8006,  0.5992,  0.0000],
        [ 0.0000, -0.5992, -0.8006,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

In [220]:
f2 = f1 @ flocal2 @ frot2

tensor([[ 0.3180,  0.3453, -0.8830,  1.8126],
        [-0.7186,  0.6953,  0.0132, -0.8578],
        [ 0.6185,  0.6303,  0.4692,  1.6893],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])

```python
RTF2[seq_i,:,:]: tensor([[ 0.3180,  0.3453, -0.8830,  1.8126],
                         [-0.7186,  0.6953,  0.0132, -0.8577],
                         [ 0.6185,  0.6303,  0.4692,  1.6894],
                         [ 0.0000,  0.0000,  0.0000,  1.0000]])
```

In [221]:
f2 @ tensor([0.5541, 1.4027, 0.000, 1])

tensor([ 2.4732, -0.2806,  2.9162,  1.0000])

In [222]:
RMSD([2.645  , 0.032,   2.778], f2 @ tensor([0.5541, 1.4027, 0.000, 1]))

0.38255755050765444